In [1]:
# Purpose:  Model topics for group 2 ( 7th republic debate)
# Author:  Preeti Javaji
# Date:  05/15/16
# Course:  MAS DSE capstone, Spring 2016

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import math

pd.set_option('display.max_colwidth', 240)
pd.set_option('display.max_columns', 50)

import sys
import os
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB


print 'done'

done


In [2]:
# Function defs.

# return string with non-ascii chars removed
def to_ascii(s):
    returnstr = s.strip()
    returnstr = "".join([ch for ch in returnstr if ord(ch)< 128])
    return returnstr

print 'done'

done


In [3]:
# Load data file & clean the 'text' column.

df = pd.read_csv("Sentiment_backup.csv")

# Convert to ascii
df['ascii'] = df['text'].apply(to_ascii)
#df['ascii'] = df['ascii'].str.encode('ascii',errors='ignore') 
df['ascii'] = df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
df['ascii'] = df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
df['ascii'] = df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
df['ascii'] = df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
df['ascii'] = df['ascii'].str.lower()

df['K_sentiment'] = df['sentiment']
print df.shape
#print df[df['ascii']!=df['text']][['ascii', 'K_sentiment']].head(10)
df[['text', 'ascii']].head(3)
#df[df['ascii'].str.contains("\n")][['text', 'ascii']].head(10)

(4333, 23)


,text,ascii
0,RT @TheBaxterBean: Scott Walker's Abortion Ban Allows Rapist Father To Sue For Emotional Distress http://t.co/rHMvgumuir #GOPDebate @Badger��_,rt @thebaxterbean scott walkers abortion ban allows rapist father to sue for emotional distress url #gopdebate @badger_
1,"RT @mydaughtersarmy: Dear GOP,\rIf you want to stop abortions... \r\r#GOPDebate \rhttp://t.co/RbL1t4iPx6","rt @mydaughtersarmy dear gop,\rif you want to stop abortions... \r\r#gopdebate \rurl"
2,RT @Bipartisanism: A message to the #GOPDebate candidates: http://t.co/WFK5CmVgiN,rt @bipartisanism a message to the #gopdebate candidates url


In [4]:
# Show the class frequencies.
counts = df['subject_matter'].value_counts()
total = counts.sum()

counts = counts.apply(lambda x: ("Count: %d" % x, "Frequency: %.2f" % (float(x)/total)) )

#print("%.2f" % a)
#string = 'string%d' % (i,)

print counts
print type(counts)

None of the above                       (Count: 1271, Frequency: 0.33)
Religion                                 (Count: 407, Frequency: 0.11)
Women's Issues (not abortion though)     (Count: 362, Frequency: 0.10)
Racial issues                            (Count: 353, Frequency: 0.09)
Abortion                                 (Count: 293, Frequency: 0.08)
Jobs and Economy                         (Count: 251, Frequency: 0.07)
Immigration                              (Count: 211, Frequency: 0.06)
FOX News or Moderators                   (Count: 188, Frequency: 0.05)
LGBT issues                              (Count: 126, Frequency: 0.03)
iran deal                                 (Count: 74, Frequency: 0.02)
Healthcare (including Medicare)           (Count: 67, Frequency: 0.02)
Gun Control                               (Count: 61, Frequency: 0.02)
Foreign Policy                            (Count: 41, Frequency: 0.01)
ISIS                                      (Count: 40, Frequency: 0.01)
none  

In [5]:
%pwd

u'/Users/preetijavaji/Documents/DSE/capstone'

In [6]:
# Add the 1st democratic debate transcript
D1_df = pd.read_csv("labeled_transcripts/R7_proc_labelled.csv")
D1_df.head()

,Unnamed: 0,duration,speaker,speaker_type,start_time,text,topic
0,0,225,BAIER,moderator,0,"BAIER: Nine p.m. on the East Coast. Eight o'clock here in Des Moines, Iowa. Welcome to the seventh Republican presidential debate of the 2016 campaign. I'm Bret Baier, along with my comoderators Megyn Kelly and Chris Wallace.",intro
1,1,244,KELLY,moderator,225,"KELLY: Tonight, we are coming to you live from the Iowa Event Center in downtown Des Moines, in the shadow of the Iowa state capitol. In just four days, people here in the Hawkeye State will caucus and cast the first votes of the 2016 e...",intro
2,2,388,WALLACE,moderator,469,"WALLACE: Tonight, we're partnering with Google for an exclusive second-screen experience. Search Fox News debate on Google any time in the next two hours to access exclusive content. You can see the campaigns respond to the debate in re...",NaN
3,3,249,BAIER,moderator,857,"BAIER: Google sees political searches spike during televised presidential debates ��� no surprise there. But they spike by more than 440 percent on average, as voters across America seek to learn more about what's happening on the live ...",NaN
4,4,50,APPLAUSE,audience,1330,[applause],NaN


In [7]:
#clean it up too
D1_df['ascii'] = D1_df['text'].apply(to_ascii)

D1_df['ascii'] = D1_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
D1_df['ascii'] = D1_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
D1_df['ascii'] = D1_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
D1_df['ascii'] = D1_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
D1_df['ascii'] = D1_df['ascii'].str.lower()

In [8]:
# Topics from the debate
D1_df['topic'] = D1_df['topic'].str.lower()
a = D1_df['topic'].unique()
topics = list(a)
topics.sort()
topics

[nan,
 'abortion',
 'belgium',
 'bi-partisanship',
 'bill clinton',
 'body cameras',
 'bush vs clinton',
 'christie vs clinton',
 'climate change',
 'endoresement',
 'energy',
 'experience',
 'federal spending',
 'ferguson',
 'flint water crisis',
 'immigration',
 'insider vs outsider',
 'intro',
 'iphone hack',
 'iran nuclear deal',
 'iraq war',
 'isis',
 'islamophobia',
 'libertarinism',
 'medicaid',
 'military strength',
 'none',
 'nsa',
 'obamacare',
 'open immigration',
 'planned parenthood',
 'privacy',
 'profiling',
 'puerto rico',
 'radical muslims',
 'religious liberty',
 'rubio',
 'rubio vs clinton',
 'russia',
 'trump ',
 'uniting gop',
 'veterans']

In [9]:
# Now read hashtag tweets
D2_df = pd.read_csv("labeled_transcripts/HashtagTweets_updated.csv")
D2_df.head()

,topic,tweetid,text
0,benghazi,7.050000e+17,"""We will never forget #Benghazi @HillaryClinton https://t.co/lkfUGTaca4 #veterans #honor #courage #commitment #SuperTuesday #VoteTrump """
1,benghazi,7.050000e+17,"""##Benghazi Heroes Endorse Donald Trump https://t.co/v6jUWZdy0m """
2,benghazi,7.050000e+17,""".@hillaryclinton Great shot! Was this before or after the imaginary snipers? #VinceFoster #Benghazi #Whitewater https://t.co/0WnW2z4GAm """
3,benghazi,7.050000e+17,"""(IBD) Lying About #Benghazi Terrorist Attack Sole Hillary Accomplishment - https://t.co/ncx9a5LEmk #PJNET https://t.co/3Z7RV3ydv5 123 """
4,benghazi,7.050000e+17,"""what about the parents of #benghazi victims you lied to? @HillaryClinton @sitflyr """


In [10]:
#clean it up
#clean it up too
D2_df['ascii'] = D2_df['text'].apply(to_ascii)

D2_df['ascii'] = D2_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
D2_df['ascii'] = D2_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
D2_df['ascii'] = D2_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
D2_df['ascii'] = D2_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
D2_df['ascii'] = D2_df['ascii'].str.lower()

In [11]:
# Now take the text and topic from both the dataframes and concat
D1_new = D1_df[['ascii','topic']]
T_DF = df[['ascii','subject_matter']]

T_DF = T_DF.rename(columns = {'subject_matter':'topic'})

In [12]:
D2_new = D2_df[['ascii','topic']]
T_new1 = pd.concat([T_DF,D1_new])
print T_new1.shape
T_new = pd.concat([T_new1,D2_new])
print T_new.shape

(4734, 2)
(6253, 2)


In [13]:
# clean the topics before encoding
# first convert to lower case
T_new['topic'] = T_new['topic'].str.lower()
a = T_new['topic'].unique()
topics = list(a)
topics.sort()
topics

[nan,
 'abortion',
 'belgium',
 'benghazi',
 'bi-partisanship',
 'bill clinton',
 'billclinton',
 'body cameras',
 'border security',
 'bordersecurity',
 'bush vs clinton',
 'christie vs clinton',
 'climate change',
 'clinton emails',
 'cnn',
 'cnndebate',
 'cnntownhall',
 'college',
 'common core',
 'endoresement',
 'energy',
 'environment',
 'experience',
 'family leave',
 'familyleave',
 'federal spending',
 'ferguson',
 'flint water crisis',
 'flintwater',
 'flintwatercrisis',
 'foreign aid',
 'foreign policy',
 'fox news or moderators',
 'gun control',
 'healthcare (including medicare)',
 'homelandsecurity',
 'immigration',
 'insider vs outsider',
 'intro',
 'iphone hack',
 'iran',
 'iran deal',
 'iran nuclear deal',
 'iraq',
 'iraq war',
 'isis',
 'islamophobia',
 'israel',
 'jobs and economy',
 'lgbt issues',
 'libertarinism',
 'marijuana',
 'medicaid',
 'military strength',
 'none',
 'none of the above',
 'nsa',
 'obama',
 'obamacare',
 'open immigration',
 'planned parenthood'

In [73]:
# After seeing the list above we will combine:
1.'abortion','planned parenthood', 
2.'bill clinton', 'billclinton', 
3.'climate change',  'environment', 'energy',
4.'cnn', 'cnndebate','cnntownhall',
5.'family leave', 'familyleave',
6.'flint water crisis', 'flintwatercrisis', 'flintwater',
7.'foreign policy',  'russia', 'foreign aid', 'homelandsecurity','nsa', 'privacy', 'military strength',
8.'healthcare (including medicare)', 'obamacare','medicaid',
9.'immigration', 'islamophobia', 'open immigration','border security','bordersecurity',
10.'iran','iran deal', 'iran nuclear deal',
11.'iraq','iraq war',
12.'isis', 'belgium', 'radical islam', 'radical muslims',
13.'jobs and economy','taxes',
14.'none','none of the above',
15.'racial issues','body cameras', 'ferguson', 'profiling',
16.'religion', 'religious liberty',
17.'social security', 'federal spending',
18.'zika','zikavirus'


In [15]:
T_new.ix[T_new['topic'] == 'planned parenthood','topic'] = 'abortion' #1
T_new.ix[T_new['topic'] == 'billclinton','topic'] = 'bill clinton'  #2
T_new.ix[T_new['topic'].isin( ['climate change','energy']),'topic'] = 'environment' #3
T_new.ix[T_new['topic'].isin( ['cnndebate','cnntownhall']),'topic'] = 'cnn'  #4
T_new.ix[T_new['topic'] == 'familyleave','topic'] = 'family leave' #5
T_new.ix[T_new['topic'].isin( ['flint water crisis','flintwatercrisis']),'topic'] = 'flintwater' #6
T_new.ix[T_new['topic'].isin( ['russia','foreign aid','homelandsecurity','nsa', 'privacy', 'military strength']),'topic'] = 'foreign policy' #7
T_new.ix[T_new['topic'].isin( ['obamacare','medicaid']),'topic'] = 'healthcare (including medicare)' #8
T_new.ix[T_new['topic'].isin( ['islamophobia', 'open immigration','border security','bordersecurity']),'topic'] = 'immigration' #9
T_new.ix[T_new['topic'].isin( ['iran deal', 'iran nuclear deal']),'topic'] = 'iran' #10
T_new.ix[T_new['topic'] == 'iraq war','topic'] = 'iraq' #11
T_new.ix[T_new['topic'].isin( ['belgium', 'radical islam', 'radical muslims']),'topic'] = 'isis' #12
T_new.ix[T_new['topic'] == 'taxes','topic'] = 'jobs and economy' #13
T_new.ix[T_new['topic'] == 'none of the above','topic'] = 'none' #14
T_new.ix[T_new['topic'].isin( ['ferguson','body cameras', 'ferguson', 'profiling']),'topic'] = 'racial issues' #15
T_new.ix[T_new['topic'] == 'federal spending','topic'] = 'social security' #16
T_new.ix[T_new['topic'] == 'zikavirus','topic'] = 'zika' #17
T_new.ix[T_new['topic'] == 'religious liberty','topic'] = 'religion' #18



In [16]:
a = T_new['topic'].unique()
topics = list(a)
topics.sort()
topics

[nan,
 'abortion',
 'benghazi',
 'bi-partisanship',
 'bill clinton',
 'bush vs clinton',
 'christie vs clinton',
 'clinton emails',
 'cnn',
 'college',
 'common core',
 'endoresement',
 'environment',
 'experience',
 'family leave',
 'flintwater',
 'foreign policy',
 'fox news or moderators',
 'gun control',
 'healthcare (including medicare)',
 'immigration',
 'insider vs outsider',
 'intro',
 'iphone hack',
 'iran',
 'iraq',
 'isis',
 'israel',
 'jobs and economy',
 'lgbt issues',
 'libertarinism',
 'marijuana',
 'none',
 'obama',
 'puerto rico',
 'racial issues',
 'religion',
 'rubio',
 'rubio vs clinton',
 'social security',
 'socialism',
 'syria',
 'torture',
 'trump ',
 'uniting gop',
 'veterans',
 "women's issues (not abortion though)",
 'zika']

In [17]:
# remove intro closing nan and flipflopping 

T_new =T_new[~T_new['topic'].isin(['intro','flipflopping','closing'])]
T_new.shape

(6248, 2)

In [18]:
# set all nans in topic to none
T_new.ix[T_new['topic'].isnull(),'topic'] = 'none'

In [19]:
T_new['topic'].value_counts()

none                                    2110
religion                                 409
women's issues (not abortion though)     362
racial issues                            358
abortion                                 298
jobs and economy                         252
immigration                              239
cnn                                      228
fox news or moderators                   188
bill clinton                             182
socialism                                137
healthcare (including medicare)          136
iran                                     135
lgbt issues                              126
isis                                     115
flintwater                               104
gun control                              103
israel                                    92
benghazi                                  85
syria                                     82
veterans                                  75
obama                                     56
iraq      

In [20]:
# Sample the data so as to get equal numbers of each class.
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(T_new['ascii'],T_new['topic'], test_size=0.3, random_state=42)


In [21]:
    
print len(X_train), len(X_test) , len(y_train) , len(y_test)

4373 1875 4373 1875


In [22]:
stop_words = pd.read_table('/Users/preetijavaji/Documents/DSE/capstone/stop_words.org',names='words')
stop_words_list = list(stop_words['w'])
from sklearn.feature_extraction.text import CountVectorizer
vectorizer   =  CountVectorizer(min_df = 1,stop_words = stop_words_list, ngram_range = (1,2))
X_f_train = vectorizer.fit_transform(X_train)

In [23]:
classifiers = []
#classifiers.append( svm.SVC() )
classifiers.append( OneVsOneClassifier(LinearSVC(random_state=0)) )
classifiers.append( OneVsRestClassifier(LinearSVC(random_state=0)) )
classifiers.append( OneVsRestClassifier(MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)) )

#classifier_1 = OneVsOneClassifier(svm.SVC())
#OneVsOneClassifier(LinearSVC(random_state=0)).fit(X, y).predict(X)

cv = 10  # Number of crossvalidation folds to use.
scores = []

for classifier in classifiers:
    scores.append( cross_validation.cross_val_score(classifier, X_f_train, y_train, cv=cv) )
    
for score in scores:
    print score
    print score.mean()
    print

print 'done'

[ 0.77056277  0.79028698  0.77303371  0.79908676  0.77880184  0.7764977
  0.79118329  0.76223776  0.81220657  0.79097387]
0.784487125271

[ 0.80519481  0.81898455  0.76404494  0.80821918  0.79953917  0.81797235
  0.80742459  0.77156177  0.81220657  0.81472684]
0.801987477445

[ 0.76839827  0.7593819   0.73483146  0.75799087  0.74884793  0.74423963
  0.76798144  0.74592075  0.77934272  0.74584323]
0.755277819056

done


/Users/preetijavaji/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:516: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)


In [24]:
XT_f = vectorizer.transform(X_test)
predicteds = []
for classifier in classifiers:
    classifier.fit(X_f_train, y_train)
    yy = classifier.predict(XT_f)
    print yy.shape
    predicteds.append( classifier.predict(XT_f))


(1875,)
(1875,)
(1875,)


In [25]:
# Apply classifiers to the whole (not sampled) dataset and look at the confusion matrix and classification report for each.

# Format the whole dataset.
X_all = T_new['ascii'].values
y_all = T_new['topic'].values


# Create feature vectors from the tweets. Use the same feature vector as above.
X_featurized_all = vectorizer.transform(X_all)

# Generate predictions using the classifier trained on the equally sampled data.
predicteds = []
for classifier in classifiers:
    predicteds.append( classifier.predict(X_featurized_all) )
    
for y_pred_all in predicteds:
    print accuracy_score(y_all, y_pred_all)
    #print '   Neg ', 'Neu ', 'Pos'
    print confusion_matrix(y_all, y_pred_all)
    print classification_report(y_all, y_pred_all)
    print


0.925896286812
[[253   0   0 ...,   0  17   0]
 [  0  82   0 ...,   1   0   0]
 [  0   0   0 ...,   0   0   0]
 ..., 
 [  0   0   0 ...,  74   0   0]
 [  4   0   0 ...,   0 330   0]
 [  0   0   0 ...,   0   0  31]]
             precision    recall  f1-score   support

   abortion       0.95      0.85      0.90       298
   benghazi       0.99      0.96      0.98        85
bi-partisanship       0.00      0.00      0.00         2
bill clinton       0.98      0.99      0.99       182
bush vs clinton       1.00      0.50      0.67         2
christie vs clinton       1.00      1.00      1.00         1
clinton emails       0.94      0.94      0.94        35
        cnn       0.95      0.99      0.97       228
    college       1.00      0.84      0.91        19
common core       1.00      1.00      1.00         1
endoresement       1.00      1.00      1.00         2
environment       1.00      0.87      0.93        38
 experience       1.00      0.67      0.80         3
family leave       1.

/Users/preetijavaji/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [27]:
# read the tweet sample into a df
Tweet_df = pd.read_table('Tweet_g2.csv',sep=',',names=['tweetid','tweet','c3','c4','c5','c6','c7','c8'])
Tweet_df

,tweetid,tweet,c3,c4,c5,c6,c7,c8
0,NaN,"""Manufacturing Day organizers recognized by Workforce Solutions Texoma – Sherman Denison Herald Democrat https://t.co/vMeVikvEBS """,Tweet_2016_1_26,691772537072717824,NaN,1453766400007,Tue Jan 26 00:00:00 +0000 2016,0
1,NaN,"""In personal exchange, @HillaryClinton laments using faith to ""judge so harshly"" https://t.co/916cWwmYjT https://t.co/aCfcAv5NK9 """,Tweet_2016_1_26,691772036230823936,NaN,1453766280000,Mon Jan 25 23:58:00 +0000 2016,14
2,6.917720e+17,"""RT @CNN: In personal exchange, @HillaryClinton laments using faith to ""judge so harshly"" https://t.co/916cWwmYjT https://t.co/aCfcAv5NK9 """,Tweet_2016_1_26,691772537127190528,NaN,1453766400020,Tue Jan 26 00:00:00 +0000 2016,0
3,NaN,"""Margaret Sanger Fan Hillary Clinton Talks Race in Ferguson https://t.co/oCEARRiTGD - American Thinker - https://t.co/2P182Perhm 119 """,Tweet_2016_1_26,691770926761971714,NaN,1453766016000,Mon Jan 25 23:53:36 +0000 2016,0
4,6.917709e+17,"""RT @gerfingerpoken2: Margaret Sanger Fan Hillary Clinton Talks Race in Ferguson https://t.co/oCEARRiTGD - American Thinker - https://t.co/… """,Tweet_2016_1_26,691772537433358336,NaN,1453766400093,Tue Jan 26 00:00:00 +0000 2016,0
5,NaN,"""An indictment for the Planned Parenthood sting videomakers https://t.co/dfNAupen6A https://t.co/add9Gex3uL """,Tweet_2016_1_26,691768644624322561,NaN,1453765471000,Mon Jan 25 23:44:31 +0000 2016,74
6,6.917686e+17,"""RT @TheAtlantic: An indictment for the Planned Parenthood sting videomakers https://t.co/dfNAupen6A https://t.co/add9Gex3uL """,Tweet_2016_1_26,691772538062573569,NaN,1453766400243,Tue Jan 26 00:00:00 +0000 2016,0
7,NaN,"""Texasu grand jury investigating Planned Parenthood videos instead indicts video creators. https://t.co/3Wqw4LhKZJ """,Tweet_2016_1_26,691767362899591169,NaN,1453765166000,Mon Jan 25 23:39:26 +0000 2016,0
8,6.917674e+17,"""RT @WBRZ: Texasu grand jury investigating Planned Parenthood videos instead indicts video creators. https://t.co/3Wqw4LhKZJ """,Tweet_2016_1_26,691772537638883328,NaN,1453766400142,Tue Jan 26 00:00:00 +0000 2016,0
9,NaN,"""We all know a dumbass named Donald Trump """,Tweet_2016_1_26,675135072048160768,NaN,1449799719000,Fri Dec 11 02:08:39 +0000 2015,25215


In [28]:
Tweet_df

,tweetid,tweet,c3,c4,c5,c6,c7,c8
0,NaN,"""Manufacturing Day organizers recognized by Workforce Solutions Texoma – Sherman Denison Herald Democrat https://t.co/vMeVikvEBS """,Tweet_2016_1_26,691772537072717824,NaN,1453766400007,Tue Jan 26 00:00:00 +0000 2016,0
1,NaN,"""In personal exchange, @HillaryClinton laments using faith to ""judge so harshly"" https://t.co/916cWwmYjT https://t.co/aCfcAv5NK9 """,Tweet_2016_1_26,691772036230823936,NaN,1453766280000,Mon Jan 25 23:58:00 +0000 2016,14
2,6.917720e+17,"""RT @CNN: In personal exchange, @HillaryClinton laments using faith to ""judge so harshly"" https://t.co/916cWwmYjT https://t.co/aCfcAv5NK9 """,Tweet_2016_1_26,691772537127190528,NaN,1453766400020,Tue Jan 26 00:00:00 +0000 2016,0
3,NaN,"""Margaret Sanger Fan Hillary Clinton Talks Race in Ferguson https://t.co/oCEARRiTGD - American Thinker - https://t.co/2P182Perhm 119 """,Tweet_2016_1_26,691770926761971714,NaN,1453766016000,Mon Jan 25 23:53:36 +0000 2016,0
4,6.917709e+17,"""RT @gerfingerpoken2: Margaret Sanger Fan Hillary Clinton Talks Race in Ferguson https://t.co/oCEARRiTGD - American Thinker - https://t.co/… """,Tweet_2016_1_26,691772537433358336,NaN,1453766400093,Tue Jan 26 00:00:00 +0000 2016,0
5,NaN,"""An indictment for the Planned Parenthood sting videomakers https://t.co/dfNAupen6A https://t.co/add9Gex3uL """,Tweet_2016_1_26,691768644624322561,NaN,1453765471000,Mon Jan 25 23:44:31 +0000 2016,74
6,6.917686e+17,"""RT @TheAtlantic: An indictment for the Planned Parenthood sting videomakers https://t.co/dfNAupen6A https://t.co/add9Gex3uL """,Tweet_2016_1_26,691772538062573569,NaN,1453766400243,Tue Jan 26 00:00:00 +0000 2016,0
7,NaN,"""Texasu grand jury investigating Planned Parenthood videos instead indicts video creators. https://t.co/3Wqw4LhKZJ """,Tweet_2016_1_26,691767362899591169,NaN,1453765166000,Mon Jan 25 23:39:26 +0000 2016,0
8,6.917674e+17,"""RT @WBRZ: Texasu grand jury investigating Planned Parenthood videos instead indicts video creators. https://t.co/3Wqw4LhKZJ """,Tweet_2016_1_26,691772537638883328,NaN,1453766400142,Tue Jan 26 00:00:00 +0000 2016,0
9,NaN,"""We all know a dumbass named Donald Trump """,Tweet_2016_1_26,675135072048160768,NaN,1449799719000,Fri Dec 11 02:08:39 +0000 2015,25215


In [29]:
# copy classifier 0 as the model for g1 and test it in spark on a sample 
# Convert to ascii
Tweet_df['ascii'] = Tweet_df['tweet'].apply(to_ascii)

# Remove hashtags
#df['ascii'] = df['ascii'].str.replace(r"#([A-Za-z0-9_]+)", " ")

# Remove handles
#df['ascii'] = df['ascii'].str.replace(r"@([A-Za-z0-9_]+)", " ")

# Remove URLs
#df['ascii'] = df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", " ")
# Replace URLs with "URL"
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")
#df['ascii'] = df['ascii'].str.replace(r"(['';:@%#()\+\*\"\…\“\”])", "")
#df['ascii'] = df['ascii'].str.replace(r"([;:@%#()\+\*\"\…\“\”])", "")

# Remove eol symbols
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbolst
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
Tweet_df['ascii'] = Tweet_df['ascii'].str.lower()


Tweet_df[['tweet', 'ascii']].head(3)
#df[df['ascii'].str.contains("\n")][['text', 'ascii']].head(10)

,tweet,ascii
0,"""Manufacturing Day organizers recognized by Workforce Solutions Texoma – Sherman Denison Herald Democrat https://t.co/vMeVikvEBS """,manufacturing day organizers recognized by workforce solutions texoma sherman denison herald democrat url
1,"""In personal exchange, @HillaryClinton laments using faith to ""judge so harshly"" https://t.co/916cWwmYjT https://t.co/aCfcAv5NK9 ""","in personal exchange, @hillaryclinton laments using faith to judge so harshly url url"
2,"""RT @CNN: In personal exchange, @HillaryClinton laments using faith to ""judge so harshly"" https://t.co/916cWwmYjT https://t.co/aCfcAv5NK9 ""","rt @cnn in personal exchange, @hillaryclinton laments using faith to judge so harshly url url"


In [30]:
X = Tweet_df['ascii'].values

#train_vectors = vectorizer.fit_transform(train_data)
#test_vectors = vectorizer.transform(test_data)
X_featurized = vectorizer.transform(X)
        
print X_featurized.shape
print 'done'

(10000, 35640)
done


In [31]:
predicted_new = []
for classifier in classifiers:
    predicted_new.append( classifier.predict(X_featurized) )

In [38]:
Tweet_df['topic'] = predicted_new[0]

In [39]:
from collections import Counter

Keys = Counter(predicted_new[0]).keys() # equals to list(set(words))
values =  Counter(predicted_new[0]).values() # counts the elements' frequency
for k,v in zip(Keys,values):
    print k,':',v

racial issues : 42
fox news or moderators : 89
marijuana : 2
college : 3
family leave : 2
lgbt issues : 8
healthcare (including medicare) : 20
religion : 26
clinton emails : 19
cnn : 195
veterans : 10
benghazi : 45
israel : 9
iran : 16
bill clinton : 273
flintwater : 10
social security : 2
immigration : 19
iraq : 3
obama : 255
none : 7369
isis : 21
gun control : 23
jobs and economy : 16
abortion : 1288
socialism : 179
syria : 1
women's issues (not abortion though) : 55


In [34]:
print len(Counter(predicted_new[0]).keys())

28


In [42]:
Tweet_df[Tweet_df['topic'] == "abortion"]['tweet']

6                            "RT @TheAtlantic: An indictment for the Planned Parenthood sting videomakers https://t.co/dfNAupen6A https://t.co/add9Gex3uL "
7                                      "Texasu grand jury investigating Planned Parenthood videos instead indicts video creators. https://t.co/3Wqw4LhKZJ "
8                            "RT @WBRZ: Texasu grand jury investigating Planned Parenthood videos instead indicts video creators. https://t.co/3Wqw4LhKZJ "
11             "Planned Parenthood hoax officially blows up: Texas grand jury decides to indict video creators instead: C... https://t.co/lPEuun6IkB #LVN "
12          "RT @lordxmen2k: Planned Parenthood hoax officially blows up: Texas grand jury decides to indict video creators instead: C... https://t.co/l… "
15                                                     "I support Planned Parenthood because I was the father in the most planned parenthood of all time. "
16                                  "RT @TheTweetOfGod: I suppor

In [43]:
# copy classifier 0 as the model for g1 and test it in spark on a sample 
import pickle
# save model in a pickle file
output = open('mymodel_g2.pkl', 'wb')
pickle.dump(classifiers[0],output)
output.close()

In [44]:
vec_file = open('myvec_g2.pkl','wb')
pickle.dump(vectorizer,vec_file)
vec_file.close()